In [ ]:
# Authors: Martin Luessi <mluessi@nmr.mgh.harvard.edu>
#          Alexandre Gramfort <alexandre.gramfort@inria.fr>
#          Nicolas P. Rougier (graph code borrowed from his matplotlib gallery)
#
# License: BSD (3-clause)

import numpy as np
import os.path as op
import os
import matplotlib.pyplot as plt
from langouEEG import *
from scipy.stats import *
from IPython.display import clear_output as clear

import mne
import pandas as pd
import pickle
from mne.datasets import sample
from mne.minimum_norm import apply_inverse_epochs, read_inverse_operator
from mne.connectivity import spectral_connectivity, envelope_correlation
from mne.viz import circular_layout, plot_connectivity_circle
import mne
from mne.datasets import eegbci
from mne.datasets import fetch_fsaverage
from mne.datasets import sample
from mne.epochs import equalize_epoch_counts
from mne.minimum_norm import make_inverse_operator, apply_inverse
from mne.minimum_norm import write_inverse_operator

import os
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D  # noqa
import mne

from lempel_ziv_complexity import lempel_ziv_complexity
from sklearn.preprocessing import MinMaxScaler

!export DISPLAY=localhost:14.0
!export MESA_GL_VERSION_OVERRIDE=3.3
import pyvista
pyvista.start_xvfb()
import matplotlib
matplotlib.use('agg')

sample_data_folder = mne.datasets.sample.data_path()
dataRoot = "/data/home/viscent/Light"
# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
isMale = False
isAll = True
isBlind = False
if not isAll:
    result_dir = op.join(dataRoot,'result','male' if isMale else 'female')
else:
    result_dir = op.join(dataRoot,'result','all')
if isBlind:
    result_dir = op.join(result_dir,'Blind')
subjects_dir = op.dirname(fs_dir)
if not op.exists(result_dir):
    os.mkdir(result_dir)
# The files live in:
subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')
print(__doc__)

In [ ]:
%%capture
epochs_4F = dict()
epochs_RF = dict()
epochs_4R = dict()
epochs_RR = dict()
for subject_name in range(1,21):
    if not isAll:
        if not (isMale ^ (subject_name in [7,8,11,17])):
            continue
    if subject_name<10:
        subject_name='S0'+str(subject_name)
    else:
        subject_name='S'+str(subject_name)
    with open(dataRoot+'/clean_data_av/'+subject_name+'_clean.lgeeg','rb') as f:
        raw=pickle.load(f)
    events, event_dict=extractEvents(raw)
    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=True,
                        exclude='bads')
    if isBlind:
        epoch_RR,epoch_RF,epoch_4R,epoch_4F = extractEpochsBlind(raw,events,picks)
    else:
        epoch_RR,epoch_RF,epoch_4R,epoch_4F = extractEpochs(raw,events,picks)
    epochs_4F[subject_name]=epoch_4F
    epochs_RF[subject_name]=epoch_RF
    epochs_RR[subject_name]=epoch_RR
    epochs_4R[subject_name]=epoch_4R
    data_path = sample.data_path()

    picks = mne.pick_types(raw.info, meg=False, eeg=True, stim=False, eog=True,
                        exclude='bads')
epoch_4F = mne.concatenate_epochs(list(epochs_4F.values()))
epoch_4R = mne.concatenate_epochs(list(epochs_4R.values()))
epoch_RF = mne.concatenate_epochs(list(epochs_RF.values()))
equalize_epoch_counts([epoch_4F,epoch_RF,epoch_4R])




In [ ]:
threshold_4F = np.expand_dims(epoch_4F.get_data().mean(axis=2),2).repeat(epoch_4F.get_data().shape[2],axis=2)
threshold_RF = np.expand_dims(epoch_RF.get_data().mean(axis=2),2).repeat(epoch_RF.get_data().shape[2],axis=2)
threshold_4R = np.expand_dims(epoch_4R.get_data().mean(axis=2),2).repeat(epoch_4R.get_data().shape[2],axis=2)

In [ ]:
bin_4F = np.where(epoch_4F.get_data()>=threshold_4F,1,0)
bin_RF = np.where(epoch_RF.get_data()>=threshold_RF,1,0)
bin_4R = np.where(epoch_4R.get_data()>=threshold_4R,1,0)

In [ ]:
str_4F = []
for i in range(bin_4F.shape[0]):
    strs = []
    for j in range(bin_4F.shape[1]):
        tstr = ''
        for k in range(bin_4F.shape[2]):
            tstr += str(bin_4F[i,j,k])
        strs.append(tstr)
    str_4F.append(strs)
str_RF = []
for i in range(bin_4F.shape[0]):
    strs = []
    for j in range(bin_4F.shape[1]):
        tstr = ''
        for k in range(bin_4F.shape[2]):
            tstr += str(bin_RF[i,j,k])
        strs.append(tstr)
    str_RF.append(strs)
str_4R = []
for i in range(bin_4F.shape[0]):
    strs = []
    for j in range(bin_4F.shape[1]):
        tstr = ''
        for k in range(bin_4F.shape[2]):
            tstr += str(bin_4R[i,j,k])
        strs.append(tstr)
    str_4R.append(strs)

In [ ]:
epochs_count = len(list(epochs_4F.values()))
epoch_len = bin_4F.shape[2]
lzc_4F,lzc_RF,lzc_4R = np.zeros((bin_4F.shape[0],bin_4F.shape[1])),np.zeros((bin_4F.shape[0],bin_4F.shape[1])),np.zeros((bin_4F.shape[0],bin_4F.shape[1]))
for i in range(bin_4F.shape[0]):
    for j in range(bin_4F.shape[1]):
        lzc_4F[i,j]=lempel_ziv_complexity(str_4F[i][j])
        lzc_RF[i,j]=lempel_ziv_complexity(str_RF[i][j])
        lzc_4R[i,j]=lempel_ziv_complexity(str_4R[i][j])

In [ ]:
# scaler = MinMaxScaler(feature_range=(0,1))
# norm_lzc_4F = scaler.fit_transform(lzc_4F)
# norm_lzc_RF = scaler.fit_transform(lzc_RF)
# norm_lzc_4R = scaler.fit_transform(lzc_4R)
# avg_lzc_4F = norm_lzc_4F.mean(axis=0)
# avg_lzc_RF = norm_lzc_RF.mean(axis=0)
# avg_lzc_4R = norm_lzc_4R.mean(axis=0)
avg_lzc_4F = lzc_4F.mean(axis=0)
avg_lzc_RF = lzc_RF.mean(axis=0)
avg_lzc_4R = lzc_4R.mean(axis=0)


In [ ]:
pd_4F = pd.DataFrame(lzc_4F)
pd_4F.to_csv('4F_TCLZC.csv', index=False, header=False)
pd_RF = pd.DataFrame(lzc_RF)
pd_RF.to_csv('RF_TCLZC.csv', index=False, header=False)
pd_4R = pd.DataFrame(lzc_4R)
pd_4R.to_csv('4R_TCLZC.csv', index=False, header=False)

In [ ]:
dlzc = avg_lzc_4F - avg_lzc_4R
vmax = np.max([avg_lzc_4F[0:64].max(),avg_lzc_4R[0:64].max(),avg_lzc_RF[0:64].max()])
vmin = np.min([avg_lzc_4F[0:64].min(),avg_lzc_4R[0:64].min(),avg_lzc_RF[0:64].min()])
print(vmax,vmin)
fig = plt.figure(dpi = 300)
plot_topomap(avg_lzc_4F[0:64],epoch_4F.info,vmin = avg_lzc_4F[1:64].min(),vmax=vmax,cmap='jet')
fig.savefig(op.join(result_dir,'4F_TCLZC.svg'),dpi=300)
fig.clf()
fig = plt.figure(dpi = 300)
plot_topomap(avg_lzc_RF[0:64],epoch_RF.info,vmin = avg_lzc_RF[1:64].min(),vmax=vmax,cmap='jet')
fig.savefig(op.join(result_dir,'RF_TCLZC.svg'),dpi=300)
fig.clf()
fig = plt.figure(dpi = 300)
plot_topomap(avg_lzc_4R[0:64],epoch_4R.info,vmin = avg_lzc_4R[1:64].min(),vmax=vmax,cmap='jet')
fig.savefig(op.join(result_dir,'4R_TCLZC.svg'),dpi=300)
fig.clf()
dlzc = avg_lzc_4F - avg_lzc_4R
plot_topomap(dlzc[0:64],epoch_4R.info,vmin = dlzc[0:64].min(),vmax=dlzc[0:64].max(),cmap='jet')
dlzc = avg_lzc_4F - avg_lzc_RF
plot_topomap(dlzc[0:64],epoch_4R.info,vmin = dlzc[0:64].min(),vmax=dlzc[0:64].max(),cmap='jet')